In [3]:
class CPU:
    
    def __init__(self, ram, counter=0):
        self.ram = [ram]
        self.counter = counter
        self.hold = None
        self.dict_indx = self.ram[0][str(self.counter)]
    
    def cycle(self, slot=0):
        '''
        Ex RAM:
        ---------------
        ADDRESS|VALUE
        ---------------
           0   |LOAD 6
           1   |ADD 7
           2   |STORE 6
           3   |JUMP 1
           4   |NUM 0
           5   |NUM 0
           6   |NUM 7
           7   |NUM 1

        LOAD -> Loads a value from a given line, i.e LOAD 6, loads the value in line 6 and sets
        it as held value 
        (held = 7)
        
        ADD -> Adds the value from a given line to the held number, i.e ADD 7 adds 1 to the held
        number
        (held = 1 + 7 = 8)
        
        STORE -> Sets the value in an address to the held number, i.e STORE 6 stores 7 to address 6 
        (ADDRESS 6|NUM 7)
        
        JUMP -> Jumps to the given address i.e, JUMP 1 moves back to address 1
        (cur_line = 1)
        
        NUM -> Used to hold values, the ticker will skip this line, i.e NUM 1 holds the number 1
        (NUM 1 = 1 "digit form")
        
        Extra:
        
        IF -> Compares 2 addresses by subtracting them, if the answer is positive, continue code, 
        move an extra step if negative
        (IF 6 7 = 6-7 = -1 -> counter += 2)
        
        RET -> Usually the lass address of a set of ram, gives model.cycle() a return value
        (RET 14 -> model.cycle() = 14)
        '''
        
        
        if self.dict_indx[0] == "LOAD": self.exc_LOAD(slot)
        
        if self.dict_indx[0] == "ADD": self.exc_ADD(slot)
        
        if self.dict_indx[0] == "STORE": self.exc_STORE(slot)
        
        if self.dict_indx[0] == "NUM": self.exc_NUM(slot)
        
        if self.dict_indx[0] == "JUMP": self.exc_JUMP(slot)
        elif self.dict_indx[0] == "IF": self.exc_IF(slot)
        else: self.counter += 1
        
        if self.dict_indx[0] == "RET":
            self.counter = 0
            return self.dict_indx[1]
                
        self.dict_indx = self.ram[slot][str(self.counter)]
        
        
        
    def run(self, slot=0):
        temp = CPU(self.ram(0))
        while(temp.cycle() == None):
            temp.cycle(slot)
        return temp.cycle(slot)
        
    #Load Function
    def exc_LOAD(self, slot):
        self.hold = self.ram[slot][str(self.dict_indx[1])][1]
        return None
        
    #Add Function
    def exc_ADD(self, slot):
        self.hold += self.ram[slot][str(self.dict_indx[1])][1]
        return None
        
    #Store Function
    def exc_STORE(self, slot):
        self.ram[slot][str(self.dict_indx[1])][1] = self.hold
        return None
    
    #Jump Function
    def exc_JUMP(self, slot):
        self.counter = self.dict_indx[1]
        return None
        
    #Num Function
    def exc_NUM(self, slot):
        return None
    
    #If Function
    def exc_IF(self, slot):
        if self.ram[slot][str(self.dict_indx[1])][1] - self.ram[slot][str(self.dict_indx[2])][1] < 0:
            self.counter += 2
        else: self.counter += 1
            
    def full_cycle(self):
        c = 0
        while(self.cycle() == None):
            c += 1
            self.cycle()
            
        return self.cycle(), c

        
        
        
        
        

In [4]:
#Below are some examples of programming with this cpu

In [9]:
val_a = 10
val_b = 12

In [10]:
multiply = {
    "0": ["LOAD", 15],
    
    #if AD 13 - AD 12 > 0
    "1": ["IF", 13, 12],
    "2": ["JUMP", 4], #If decum is positive
    "3": ["JUMP", 10], #If decum is negative
    #if pos
    "4": ["ADD", 11], #(0 + 3, 3 + 3)
    "5": ["STORE", 15],
    "6": ["LOAD", 13],
    "7": ["ADD", 14], #(1-0 = 0, 0-1 = -1)
    "8": ["STORE", 13],
    "9": ["JUMP", 0],
    
    #if neg
    "10": ["JUMP", 15],
    
    #vars
    "11": ["NUM", val_a], #var1
    "12": ["NUM", 1], #var2 = 1 (Takes difference of address 13)
    "13": ["NUM", val_b], #var3 (times multiplied by var1)
    "14": ["NUM", -1], #var4 = -1 (number decumulated by)
    "15": ["RET", 0]  #RETURN address (3, 6)
}

In [11]:
model = CPU(multiply)

In [12]:
ans = model.full_cycle()

print(f"Took {ans[1]} runs to finish execution, result is {ans[0]}")

Took 56 runs to finish execution, result is 120
